<a href="https://colab.research.google.com/github/sugiyama404/chat_bot_bert/blob/main/bert_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!git clone https://github.com/huggingface/transformers
#%cd transformers
#!pip install .

In [2]:
from google.colab import drive 
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/Colab Notebooks/workspace/transformers"
!pip install tokenizers sacremoses pyyaml huggingface-hub transformers --use-feature=in-tree-build

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/workspace/transformers
Processing /content/drive/MyDrive/Colab Notebooks/workspace/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 7.6 MB/s 
     |████████████████████████████████| 636 kB 50.4 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 895 kB 60.0 MB/s 
  Created wheel for transformers: filename=transformers-4.11.0.dev0-py3-none-any.whl size=2870160 sha256=c2c34528b5bf800b5ffe4ca70a1688fed20d9d110be1bfca38dd8fd48bd2b36e
  Stored in directory: /tmp/pip-ephem-wheel-cache-j921q86b/wheels/40/8a/59/e0ddc79685e8e4d608b414fb108a351e05086e4f0af4ec3774
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing ins

In [3]:
!pip install fugashi[unidic-lite] ipadic datasets

     |████████████████████████████████| 490 kB 7.1 MB/s 
     |████████████████████████████████| 13.4 MB 25.5 MB/s 
     |████████████████████████████████| 270 kB 53.7 MB/s 
     |████████████████████████████████| 243 kB 47.3 MB/s 
     |████████████████████████████████| 1.3 MB 53.2 MB/s 
     |████████████████████████████████| 123 kB 56.3 MB/s 
     |████████████████████████████████| 294 kB 57.6 MB/s 
     |████████████████████████████████| 142 kB 66.3 MB/s 
     |████████████████████████████████| 47.4 MB 74 kB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=208c31bd5a3c1eb5b36a569072e3cf756be00da8f2bf3034040334365ad8a489
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=9abdf20db9bedc0235592040e2de15662574b9e9e6cca708b070861c06498285
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/

# モデル構築

In [4]:
'''
!python ./examples/legacy/question-answering/run_squad.py \
    --model_type=bert \
    --model_name_or_path=cl-tohoku/bert-base-japanese-whole-word-masking \
    --do_train \
    --do_eval \
    --train_file=DDQA-1.0_RC-QA_train.json \
    --predict_file=DDQA-1.0_RC-QA_dev.json \
    --per_gpu_train_batch_size 12 \
    --learning_rate 3e-5 \
    --num_train_epochs 10 \
    --max_seq_length 384 \
    --doc_stride 128 \
    --overwrite_output_dir \
    --output_dir output/
'''

'\n!python ./examples/legacy/question-answering/run_squad.py     --model_type=bert     --model_name_or_path=cl-tohoku/bert-base-japanese-whole-word-masking     --do_train     --do_eval     --train_file=DDQA-1.0_RC-QA_train.json     --predict_file=DDQA-1.0_RC-QA_dev.json     --per_gpu_train_batch_size 12     --learning_rate 3e-5     --num_train_epochs 10     --max_seq_length 384     --doc_stride 128     --overwrite_output_dir     --output_dir output/\n'

In [16]:
from transformers import BertJapaneseTokenizer, AutoModelForQuestionAnswering
import torch

model = AutoModelForQuestionAnswering.from_pretrained('output/')  
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking') 

def reply(context, question):

    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]
    output = model(**inputs)
    answer_start = torch.argmax(output.start_logits)  
    answer_end = torch.argmax(output.end_logits) + 1 
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    answer = answer.replace(' ', '')
    return answer

In [17]:
context  = '私の名前は、タチコマです。私は公安9課に所属しています。'\
'思考戦車とは人工ニューロチップを用いた人工知能（AI）を搭載し、自ら「思考」し自律的に行動することができる戦車'

question = "あなたの名前は？"
answer = reply(context, question)

print("question: " + question)
print("answer: " + answer)

question = "あなたの所属は？"
answer = reply(context, question)

print("question: " + question)
print("answer: " + answer)

question = "思考戦車とは何ですか？"
answer = reply(context, question)

print("question: " + question)
print("answer: " + answer)

question: あなたの名前は？
answer: タチコマ
question: あなたの所属は？
answer: 公安9課
question: 思考戦車とは何ですか？
answer: 人工ニューロチップを用いた人工知能(AI)を搭載し、自ら「思考」し自律的に行動することができる戦車
